# 6.3. 填充和步幅
假设输入形状为$n_h\times n_w$，卷积核形状为$k_h\times k_w$，那么输出形状将是$(n_h-k_h+1) \times (n_w-k_w+1)$。因此，卷积的输出形状取决于输入形状和卷积核的形状。
有时，在应用了连续的卷积之后，我们最终得到的输出远小于输入大小。这是由于卷积核的宽度和高度通常大于$1$所导致的。比如，一个$240 \times 240$像素的图像，经过$10$层$5 \times 5$的卷积后，将减少到$200 \times 200$像素。如此一来，**原始图像的边界丢失了许多有用信息**。而**填充**是解决此问题最有效的方法。

有时，我们可能希望大幅降低图像的宽度和高度。例如，如果我们发现原始的输入分辨率十分冗余。**步幅**则可以在这类情况下提供帮助。

## 6.3.1. 填充
如上所述，在应用多层卷积时，我们常常丢失边缘像素。由于我们通常使用小卷积核，因此对于任何单个卷积，我们可能只会丢失几个像素。但随着我们应用许多连续卷积层，累积丢失的像素数就多了。解决这个问题的简单方法即为**填充**（padding）：在输入图像的边界填充元素（通常填充元素是$0$）。
![](../img/6_2.png)

卷积神经网络中卷积核的高度和宽度通常为奇数，例如1、3、5或7。选择奇数的好处是，保持空间维度的同时，我们可以在顶部和底部填充相同数量的行，在左侧和右侧填充相同数量的列。

In [1]:
"""
创建一个高度和宽度为3的二维卷积层，并在所有侧边填充1个像素。给定高度和宽度为8的输入，则输出的高度和宽度也是8。
"""
import torch
from torch import nn

# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def com_conv2d(conv2d,X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X=X.reshape((1,1)+X.shape)
    Y=conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=3,padding=1)
X=torch.rand(size=(8,8))
com_conv2d(conv2d,X).shape

torch.Size([8, 8])

In [2]:
"""
当卷积核的高度和宽度不同时，我们可以填充不同的高度和宽度，使输出和输入具有相同的高度和宽度。
在如下示例中，我们使用高度为5，宽度为3的卷积核，高度和宽度两边的填充分别为2和1。
"""
conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=(5,3),padding=(2,1))
com_conv2d(conv2d,X).shape

torch.Size([8, 8])

## 6.3.2. 步幅
在计算互相关时，卷积窗口从输入张量的左上角开始，向下、向右滑动。 在前面的例子中，我们默认每次滑动一个元素。 但是，有时候**为了高效计算或是缩减采样次数**，卷积窗口可以跳过中间位置，每次滑动多个元素。我们将每次滑动元素的数量称为**步幅**（stride）。

![](../img/6_3.png)

In [3]:
"""
将高度和宽度的步幅设置为2，从而将输入的高度和宽度减半。
"""
conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=3,padding=1,stride=2)
com_conv2d(conv2d,X).shape

torch.Size([4, 4])

In [4]:
# 稍微复杂的例子。
conv2d=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=(3,5),padding=(0,1),stride=(3,4))
com_conv2d(conv2d,X).shape

torch.Size([2, 2])

为了简洁起见，当输入高度和宽度两侧的填充数量分别为$p_h$和$p_w$时，我们称之为填充$(p_h, p_w)$。当$p_h = p_w = p$时，填充是$p$。同理，当高度和宽度上的步幅分别为$s_h$和$s_w$时，我们称之为步幅$(s_h, s_w)$。特别地，当$s_h = s_w = s$时，我们称步幅为$s$。默认情况下，填充为0，步幅为1。在实践中，我们很少使用不一致的步幅或填充，也就是说，我们通常有$p_h = p_w$和$s_h = s_w$。

## 6.3.3. 小结
- 填充可以增加输出的高度和宽度。这常用来使输出与输入具有相同的高和宽。
- 步幅可以减小输出的高和宽，例如输出的高和宽仅为输入的高和宽的1/n（n是一个大于1的整数）。
- 填充和步幅可用于有效地**调整数据的维度**。